# **HUNGARIAN ALGORITHM**
***
The **Hungarian Algorithm** (also known as the Munkres-Kuhn Algorithm) is a combinatorial optimization algorithm for solving the **assignment problem**. The assignment problem involves finding the most optimal way to pair items from two sets (e.g., workers to tasks) such that the total cost is minimized (or profit maximized) while ensuring a one-to-one assignment.

The algorithm works on a cost matrix, where each entry represents the cost of assigning a specific task to a specific worker.

The assignment problem is equivalent to finding a maximum-weight matching in a weighted bipartite graph. In the context of a bipartite graph:
* One set of nodes represents workers.
* The other set represents tasks.
* Edges between nodes represent possible assignments, with weights as the cost (or profit).

In a weighted bipartite graph, an equality graph is a subgraph where the edges satisfy a specific equality condition derived from a pair of labeling functions on the vertex sets. 
\(l(u)+l(v)=w(u,v)\)
*** 
- Input: A **cost matrix** \( C \) of size \( nxn \), where \( C[u][v] \) is the cost of assigning node \( u \) in \( U \) to node \( v \) in \( V \). 
- Output: A **perfect matching** \( M \) (set of pairs \((u, v)\)) with minimum total cost.



In [2]:
import numpy as np

class HungarianAlgorithmGraph:
    def __init__(self, cost_matrix):
        """
        Initialize the HungarianAlgorithmGraph with a cost matrix.
        param cost_matrix: A square matrix where cost_matrix[u][v] represents the cost of assigning u to v.
        """
        self.cost_matrix = cost_matrix  # The cost matrix for the assignment problem
        self.n = len(cost_matrix)  # Number of nodes in each set (U and V)
        self.U = set(range(self.n))  # Set of nodes in U
        self.V = set(range(self.n))  # Set of nodes in V
        self.matching = {}  # Dictionary to store the current matching
        self.labels_u = [0] * self.n  # Labels for nodes in U
        self.labels_v = [0] * self.n  # Labels for nodes in V
        self.slack = [float('inf')] * self.n  # Slack values for nodes in V

    def initialize_labels(self):
        """
        Initialize labels for U and V.
        Labels for U are set to the maximum cost in each row of the cost matrix,
        and labels for V are initialized to 0.
        """
        for u in range(self.n):
            self.labels_u[u] = max(self.cost_matrix[u])  # Max weight in row u for label_u[u]

    def find_augmenting_path(self, u, visited_u, visited_v, parent):
        """
        Attempt to find an augmenting path starting from node u in U.
        -param u: Current node in U
        -param visited_u: List tracking visited nodes in U
        -param visited_v: List tracking visited nodes in V
        -param parent: Parent array to reconstruct paths        """
        visited_u[u] = True  # Mark the current node in U as visited
        for v in range(self.n):  # Iterate over nodes in V
            if visited_v[v]:  # Skip if node v is already visited
                continue
            # Compute the reduced cost for edge (u, v)
            delta = self.labels_u[u] + self.labels_v[v] - self.cost_matrix[u][v]
            if delta == 0:  # If edge (u, v) is in the equality graph
                visited_v[v] = True  # Mark v as visited
                # If v is unmatched or we can find an augmenting path from its match
                if v not in self.matching or self.find_augmenting_path(self.matching[v], visited_u, visited_v, parent):
                    self.matching[v] = u  # Match u with v
                    return True
            else:
                # Update the slack for node v
                self.slack[v] = min(self.slack[v], delta)
        return False  # No augmenting path found

    def update_labels(self, visited_u, visited_v):
        """
        Update labels for U and V to reduce slack and maintain feasibility.
        -param visited_u: List tracking visited nodes in U
        -param visited_v: List tracking visited nodes in V
        """
        # Compute the smallest slack value
        delta = min(self.slack[v] for v in range(self.n) if not visited_v[v])
        # Update labels for visited nodes in U and V
        for u in range(self.n):
            if visited_u[u]:
                self.labels_u[u] -= delta
        for v in range(self.n):
            if visited_v[v]:
                self.labels_v[v] += delta
            else:
                self.slack[v] -= delta

    def augment(self):
        """
        Perform augmentation for each node in U to construct the optimal matching.
        """
        for u in range(self.n):  # Iterate over all nodes in U
            visited_u = [False] * self.n  # Reset visited for U
            visited_v = [False] * self.n  # Reset visited for V
            self.slack = [float('inf')] * self.n  # Reset slack values
            # Repeat until an augmenting path is found for u
            while not self.find_augmenting_path(u, visited_u, visited_v, {}):
                self.update_labels(visited_u, visited_v)  # Update labels if no path is found
                visited_u = [False] * self.n  # Reset visited for U
                visited_v = [False] * self.n  # Reset visited for V

    def solve(self):
        """
        Solve the assignment problem using the Hungarian algorithm.
        return: A list of optimal assignments and the total cost.
        """
        self.initialize_labels()  # Step 1: Initialize labels
        self.augment()  # Step 2: Find optimal matching through augmentation
        # Extract the resulting matching
        result = [(self.matching[v], v) for v in self.matching]
        # Compute the total cost of the matching
        optimal_cost = sum(self.cost_matrix[u][v] for u, v in result)
        return result, optimal_cost

# **Example of usage**

In [3]:
cost_matrix = np.array([
    [4, 3, 2],  # Costs for assignments of u_0
    [2, 1, 5],  # Costs for assignments of u_1
    [3, 2, 4]   # Costs for assignments of u_2
])

# Instantiate the HungarianAlgorithmGraph class
hungarian = HungarianAlgorithmGraph(cost_matrix)
assignment, cost = hungarian.solve()

# Output the results
print("Optimal Assignment:", assignment)  # Matching pairs (u, v)
print("Optimal Cost:", cost)  # Total cost of the optimal matching


Optimal Assignment: [(2, 0), (1, 2), (0, 1)]
Optimal Cost: 11


In [4]:
cost_matrix = np.random.randint(0, 101, (200, 200))
hungarian = HungarianAlgorithmGraph(cost_matrix)
matching, optimal_cost = hungarian.solve()
# Displaying the size of the matching and optimal cost (since output might be large)
len(matching), optimal_cost

(200, 19932)

# Hungarian Algorithm Example

We solve the minimization assignment problem using the Hungarian Algorithm for the following cost matrix \( C \):

\[
C = 
\begin{bmatrix}
4 & 2 & 8 \\
2 & 3 & 7 \\
3 & 1 & 6
\end{bmatrix}
\]

---

### Step 1: Subtract Row Minimums
For each row, subtract the smallest value in that row from all elements in the row.

- Row 1: Minimum = 2 → \( [4-2, 2-2, 8-2] = [2, 0, 6] \)  
- Row 2: Minimum = 2 → \( [2-2, 3-2, 7-2] = [0, 1, 5] \)  
- Row 3: Minimum = 1 → \( [3-1, 1-1, 6-1] = [2, 0, 5] \)  

Resulting matrix:
\[
\begin{bmatrix}
2 & 0 & 6 \\
0 & 1 & 5 \\
2 & 0 & 5
\end{bmatrix}
\]

---

### Step 2: Subtract Column Minimums
For each column, subtract the smallest value in that column from all elements in the column.

- Column 1: Minimum = 0 → \( [2-0, 0-0, 2-0] = [2, 0, 2] \)  
- Column 2: Minimum = 0 → \( [0-0, 1-0, 0-0] = [0, 1, 0] \)  
- Column 3: Minimum = 5 → \( [6-5, 5-5, 5-5] = [1, 0, 0] \)  

Resulting matrix:
\[
\begin{bmatrix}
2 & 0 & 1 \\
0 & 1 & 0 \\
2 & 0 & 0
\end{bmatrix}
\]

---

### Step 3: Cover Zeros with Minimum Lines
Cover all zeros in the matrix using the minimum number of horizontal or vertical lines.

1. Cover Row 1 (contains a 0).  
2. Cover Column 3 (contains remaining 0s).  

Number of lines = 2, which is less than \( n = 3 \). Proceed to the next step.

---

### Step 4: Adjust the Matrix
Find the smallest uncovered element (\( \delta = 1 \)) and adjust the matrix:

- Subtract \( \delta \) from all uncovered elements.
- Add \( \delta \) to elements covered twice (intersection of two lines).

\[
\text{Resulting matrix: }
\begin{bmatrix}
1 & 0 & 1 \\
0 & 0 & 0 \\
1 & 0 & 0
\end{bmatrix}
\]

---

### Step 5: Cover Zeros Again
Cover all zeros with minimum lines.

1. Cover Column 2.  
2. Cover Column 3.  

Number of lines = 3, which equals \( n = 3 \). Proceed to the next step.

---

### Step 6: Make Assignments
Assign tasks based on zeros in the matrix, ensuring each row and column has one assignment.

1. Assign Row 1 → Column 2.  
2. Assign Row 2 → Column 3.  
3. Assign Row 3 → Column 1.  

---

### Step 7: Calculate Total Cost
The optimal assignment is:

- Task 1 → Agent 2: Cost = 2  
- Task 2 → Agent 3: Cost = 7  
- Task 3 → Agent 1: Cost = 3  

**Total Cost**: \( 2 + 7 + 3 = 12 \)

---

This completes the Hungarian Algorithm for the given cost matrix.
